<a href="https://colab.research.google.com/github/Habsie/Velib-et-Grand-Paris-Express/blob/main/stations_gpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Préparation des stations du Grand Paris Express

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyproj 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 5.1 MB/s 


In [ ]:
import pandas as pd

# Import des fichiers

In [ ]:
stations_gpe = pd.read_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/gares_gpe.csv',sep=';')

In [ ]:
gpe_shape = pd.read_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/GPE_GARE_LOCALISATION.csv')

In [ ]:
stations_velib = pd.read_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/stations-velib.csv')

In [ ]:
coor_villes = pd.read_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/repertoire_geeographique_des_communes_d_ile-de-france.csv',sep=';')

# Traitement

## Projection des coorsonnées en latitute et longitude

### Fonction pour la projection des coordonnées

In [ ]:
#Fonction projection

from pyproj import Proj, transform
def projection(x1,y1):
  inProj = Proj(init='epsg:3949')
  outProj = Proj(init='epsg:4326')
  x1,y1 = x1,y1
  x2,y2 = transform(inProj,outProj,x1,y1)
  return x2,y2

### Calcul des nouvelles coordonnées dans gpe shape

In [ ]:
#calcul des nouvelles coordonnées
gpe_shape['lon_lat'] = gpe_shape.apply(lambda gpe_shape: projection(gpe_shape['x'],gpe_shape['y']), axis=1)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/pyt

### Nettoyage des nouvelles coordonées

In [ ]:
gpe_shape['lon_lat'] = gpe_shape['lon_lat'].astype(str)

In [ ]:

gpe_shape['lon_lat'] = gpe_shape['lon_lat'].str.replace('(','')
gpe_shape['lon_lat'] = gpe_shape['lon_lat'].str.replace(')','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
gpe_shape[['longitude','latitude']] = gpe_shape['lon_lat'].str.split(',',expand=True)

In [ ]:
gpe_shape['longitude'] = pd.to_numeric(gpe_shape['longitude'])
gpe_shape['latitude'] = pd.to_numeric(gpe_shape['latitude'])

## Calcul des velib à proximité

### Fonction Calcul des velibs

In [ ]:
import geopy.distance
import math

def stations_rayon_300m(lat1,lon1):
#initialisation à chaque appel de la fonction
  nb_stations = 0
  capacity_tot = 0

  for station in range(len(stations_velib)):
  
    lat2 = stations_velib.iloc[station,4]
    lon2 = stations_velib.iloc[station,5]
    capacity = stations_velib.iloc[station,3]

    

    coords_1 = (lat1,lon1)
    coords_2 = (lat2,lon2)

    dist = geopy.distance.distance(coords_1, coords_2).m
    
    if dist < 300:
      nb_stations = nb_stations + 1
      capacity_tot = capacity_tot + capacity

  return nb_stations,capacity_tot

### Création des colonnes 'nb_stations_velib_300m' et 'nb_bornes_300m'

---



In [ ]:
gpe_shape['result'] = gpe_shape.apply( lambda gpe_shape: stations_rayon_300m(gpe_shape['latitude'],gpe_shape['longitude']),axis=1)

In [ ]:
gpe_shape['result'] = gpe_shape['result'].astype(str)
gpe_shape['result'] = gpe_shape['result'].str.replace('(','')
gpe_shape['result'] = gpe_shape['result'].str.replace(')','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
gpe_shape[['nb_stations_velib_300m','nb_bornes_300m']] = gpe_shape['result'].str.split(',',expand=True)

In [ ]:
gpe_shape['nb_stations_velib_300m'] = pd.to_numeric(gpe_shape['nb_stations_velib_300m'])
gpe_shape['nb_bornes_300m'] = pd.to_numeric(gpe_shape['nb_bornes_300m'])

## Merge

In [ ]:
stations_gpe = stations_gpe.merge(gpe_shape[['CODE','latitude','longitude','nb_stations_velib_300m','nb_bornes_300m']],left_on='Code', right_on='CODE')

# Ajout de la ville pour chaque station du gpe

## Préparation de coor_ville

In [ ]:
coor_villes['COM'] = coor_villes['COM'].astype(str).str.zfill(3)

In [ ]:
coor_villes['COM'] = coor_villes['COM'].astype(str)

In [ ]:
coor_villes['DEP'] = coor_villes['DEP'].astype(str)

In [ ]:
coor_villes['Code Commune'] = coor_villes['DEP'] + coor_villes['COM']

In [ ]:
#LISTE DES DEPARTEMENTS D'IDF NECESSAIRES POUR LE NETTOYAGE DES VILLES EN DOUBLONS
liste_depart =['75','77','78','91','92','93','94','95']

In [ ]:
#FILTRE DES VILLES PAR DEPARTEMENT POUR CONSERVER LES BONNES VILLES
coor_villes = coor_villes.loc[coor_villes['DEP'].isin(liste_depart)==True]

In [ ]:
coor_villes = coor_villes[['Code Commune','NOM','WGS84']]

## Fonction ville la plus proche

In [ ]:
import geopy.distance
import math

def ville_proche(lat1,lon1):
#initialisation à chaque appel de la fonction
  distance_courte = 10000
  for station in range(len(coor_villes)):
  
    #lat2,lon2 = coor_villes.iloc[station,2]
    #lon2 = stations_velib.iloc[station,5]
    codeinsee = coor_villes.iloc[station,0]    

    coords_1 = (lat1,lon1)
    coords_2 = (coor_villes.iloc[station,2])

    dist = geopy.distance.distance(coords_1, coords_2).km
    
    if dist < distance_courte:
      distance_courte = dist
      code = codeinsee

  return code

## Ajout de la colonne code commune

In [ ]:
stations_gpe['Code commune'] = stations_gpe.apply( lambda stations_gpe: ville_proche(stations_gpe['latitude'],stations_gpe['longitude']),axis=1)

# Nettoyage des colonnes inutiles

In [ ]:
stations_gpe = stations_gpe.drop(['code interne','CODE'],axis=1)

# Extraction

In [ ]:
stations_gpe.to_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/stations-gpe.csv',index=False)